In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [3]:
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [4]:
#examine the data
# school_data.head()

In [5]:
#examine the data
# student_data.head()

In [6]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [7]:
# assign summary data to variables
schools_total = school_data_complete["School ID"].nunique()
student_total = school_data_complete["Student ID"].nunique()
total_budget = school_data_complete["budget"].unique().sum()
average_math = school_data_complete["math_score"].mean(axis=0)
average_reading = school_data_complete["reading_score"].mean(axis=0)
passing_math_count = school_data_complete.math_score[(school_data_complete.math_score  >= 70)].count()
passing_math_percent = passing_math_count / student_total * 100
passing_reading_count = school_data_complete.reading_score[(school_data_complete.reading_score >= 70)].count()
passing_reading_percent = passing_reading_count / student_total * 100 
passing_both_count = len(school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70) ])
passing_both_percent = (passing_both_count / student_total) * 100


In [8]:
# create summary dataframe
combined_df = pd.DataFrame({"Total Schools": [schools_total],
                            "Total Students": [student_total],
                            "Total Budget":   [total_budget], # is this value correct?
                            "Average Math Score": [f'{round(average_math,1)}%'],
                            "Average Reading Score": [f'{round(average_reading, 1)}%'],
                            "Students Passing Math": [f'{round(passing_math_percent, 1)}%'],
                            "Students Passing Reading": [f'{round(passing_reading_percent, 1)}%'],
                            "Overall Passing": [f'{round(passing_both_percent, 1)}%']
                           
                           })
# formatting
combined_df['Total Budget'] = combined_df['Total Budget'].astype(int).apply(lambda x: "${:,}".format(x))
combined_df['Total Students'] = combined_df['Total Students'].astype(int).apply(lambda x: "{:,}".format(x))

# show df
combined_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Students Passing Math,Students Passing Reading,Overall Passing
0,15,"39,170","$24,649,428",79.0%,81.9%,75.0%,85.8%,65.2%


In [9]:
# Group by schools
groupby_school = school_data_complete.groupby(['school_name'])



# Create series for df
school_type = groupby_school['type'].unique()
size = groupby_school['size'].unique()
budget_per_school = groupby_school['budget'].mean()
average_math_per_school = groupby_school['math_score'].mean() 
average_reading_per_school = groupby_school['reading_score'].mean()
passing_math = school_data_complete.loc[school_data_complete['math_score'] >= 70, ['school_name','math_score']]
passing_math_counts = passing_math['school_name'].value_counts()
passing_reading = school_data_complete.loc[school_data_complete['reading_score'] >= 70, ['school_name','reading_score']]
passing_reading_counts = passing_reading['school_name'].value_counts()
passing_both = school_data_complete.loc[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70), ['school_name','math_score']]
passing_both_counts = passing_both['school_name'].value_counts()
passing_math_counts = passing_math_counts.sort_index()
passing_reading_counts = passing_reading_counts.sort_index()
passing_both_counts = passing_both_counts.sort_index()
print(passing_both_counts)

Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
Name: school_name, dtype: int64


In [10]:
schools_summary_df = pd.concat([school_type, size, budget_per_school, average_math_per_school, average_reading_per_school], axis = 1)

In [11]:
schools_summary_df['Passing Math Percent'] = round(((passing_math_counts / size) * 100).astype(float), 1)

In [12]:
schools_summary_df['Passing Reading Percent'] = round(((passing_reading_counts / size) * 100).astype(float), 1)

In [13]:
schools_summary_df['Passing Both Percent'] = round(((passing_both_counts / size) * 100).astype(float), 1)

schools_summary_df['type'] = schools_summary_df['type'].str[0]
schools_summary_df['size'] = schools_summary_df['size'].astype(int)
schools_summary_df['Budget per student'] = schools_summary_df['budget'] / schools_summary_df['size']
schools_summary_df

,type,size,budget,math_score,reading_score,Passing Math Percent,Passing Reading Percent,Passing Both Percent,Budget per student
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,77.048432,81.033963,66.7,81.9,54.6,628.0
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,94.1,97.0,91.3,582.0
Figueroa High School,District,2949,1884411,76.711767,81.158020,66.0,80.7,53.2,639.0
Ford High School,District,2739,1763916,77.102592,80.746258,68.3,79.3,54.3,644.0
Griffin High School,Charter,1468,917500,83.351499,83.816757,93.4,97.1,90.6,625.0
Hernandez High School,District,4635,3022020,77.289752,80.934412,66.8,80.9,53.5,652.0
Holden High School,Charter,427,248087,83.803279,83.814988,92.5,96.3,89.2,581.0
Huang High School,District,2917,1910635,76.629414,81.182722,65.7,81.3,53.5,655.0
Johnson High School,District,4761,3094650,77.072464,80.966394,66.1,81.2,53.5,650.0


In [14]:
# new dfs sorted by passing %
top_five = schools_summary_df.sort_values('Passing Both Percent', ascending = False)
top_five.head()

,type,size,budget,math_score,reading_score,Passing Math Percent,Passing Reading Percent,Passing Both Percent,Budget per student
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,94.1,97.0,91.3,582.0
Thomas High School,Charter,1635,1043130,83.418349,83.848930,93.3,97.3,90.9,638.0
Griffin High School,Charter,1468,917500,83.351499,83.816757,93.4,97.1,90.6,625.0
Wilson High School,Charter,2283,1319574,83.274201,83.989488,93.9,96.5,90.6,578.0
Pena High School,Charter,962,585858,83.839917,84.044699,94.6,95.9,90.5,609.0


In [15]:
# new df sorted be lowest passing #
bottom_five = schools_summary_df.sort_values('Passing Both Percent', ascending = True)
bottom_five.head()

,type,size,budget,math_score,reading_score,Passing Math Percent,Passing Reading Percent,Passing Both Percent,Budget per student
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,76.842711,80.744686,66.4,80.2,53.0,637.0
Figueroa High School,District,2949,1884411,76.711767,81.158020,66.0,80.7,53.2,639.0
Hernandez High School,District,4635,3022020,77.289752,80.934412,66.8,80.9,53.5,652.0
Huang High School,District,2917,1910635,76.629414,81.182722,65.7,81.3,53.5,655.0
Johnson High School,District,4761,3094650,77.072464,80.966394,66.1,81.2,53.5,650.0


In [22]:
# series sorted by grade (math)
ninth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "9th", ['school_name', 'math_score']]
tenth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "10th", ['school_name', 'math_score']]
eleventh_grade_math = school_data_complete.loc[school_data_complete['grade'] == "11th", ['school_name', 'math_score']]
twelfth_grade_math = school_data_complete.loc[school_data_complete['grade'] == "12th", ['school_name', 'math_score']]

In [23]:
# group series by school name (math)
ninth_grade_math = ninth_grade_math.groupby('school_name').mean()
tenth_grade_math = tenth_grade_math.groupby('school_name').mean()
eleventh_grade_math = eleventh_grade_math.groupby('school_name').mean()
twelfth_grade_math = twelfth_grade_math.groupby('school_name').mean()


In [25]:
# combine series in to df (math)

math_score_by_grade_df = pd.DataFrame({'9th': ninth_grade_math['math_score'],
                                      '10th': tenth_grade_math['math_score'],
                                       '11th': eleventh_grade_math['math_score'],
                                       '12th': twelfth_grade_math['math_score']
                                      })

math_score_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [31]:
# series sorted by grade (reading)
ninth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "9th", ['school_name', 'reading_score']]
tenth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "10th", ['school_name', 'reading_score']]
eleventh_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "11th", ['school_name', 'reading_score']]
twelfth_grade_reading = school_data_complete.loc[school_data_complete['grade'] == "12th", ['school_name', 'reading_score']]

In [33]:
# group series by school name (reading)
ninth_grade_reading = ninth_grade_reading.groupby('school_name').mean()
tenth_grade_reading = tenth_grade_reading.groupby('school_name').mean()
eleventh_grade_reading = eleventh_grade_reading.groupby('school_name').mean()
twelfth_grade_reading = twelfth_grade_reading.groupby('school_name').mean()

In [34]:
# combine series in to df (reading)

reading_score_by_grade_df = pd.DataFrame({'9th': ninth_grade_reading['reading_score'],
                                      '10th': tenth_grade_reading['reading_score'],
                                       '11th': eleventh_grade_reading['reading_score'],
                                       '12th': twelfth_grade_reading['reading_score']
                                      })

reading_score_by_grade_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
